In [ ]:
import torch
import torch.nn as nn
import cv2
import numpy as np
import sys
from PIL import Image
import matplotlib.pyplot as plt
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from my_utils import *
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import matplotlib.pyplot as plt
from scipy import ndimage
import timm
from my_utils import plot_durations
import warnings
import os
import torch.nn.functional as F
from imblearn.over_sampling import SMOTE
from collections import Counter
warnings.filterwarnings("ignore")
from model_list.model import NetworkNvidia
import torchvision.transforms as T

In [ ]:
seed = 2024
deterministic = True

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
if deterministic:
	torch.backends.cudnn.deterministic = True
	torch.backends.cudnn.benchmark = False
device=torch.device('cuda' if torch.cuda.is_available() else "cpu")

In [ ]:
CFG={
    'device': torch.device('cuda' if torch.cuda.is_available() else "cpu"),
    'EPOCH':1000,
    'BATCH_SIZE':64
    
}

In [ ]:
# total_length = 0
# for dir_path in os.listdir('expert_data'):
#     with open(os.path.join('expert_data', dir_path, 'len.txt')) as f:
#         total_length += int(f.readline())
# print("TOTAL_LENGTH: ",total_length)
# expert_states = np.empty((total_length, 9, 112, 256), dtype=np.float32)
# expert_commands = np.empty((total_length, 3), dtype=np.float32)
# expert_speeds = np.empty((total_length, 1), dtype=np.float32)
# expert_actions = np.empty((total_length, 3), dtype=np.float32)
# loaded_length = 0
# for dir in tqdm(os.listdir('expert_data')):
#     np_states = np.load(os.path.join('expert_data', dir, 'expert_states.npy'))
#     np_commands = np.load(os.path.join('expert_data', dir, 'expert_commands.npy'))
#     np_speeds = np.load(os.path.join('expert_data', dir, 'expert_speeds.npy'))
#     np_actions = np.load(os.path.join('expert_data', dir, 'expert_actions.npy'))

#     episode_length = np_states.shape[0]
#     expert_states[loaded_length: loaded_length + episode_length] = np_states
#     expert_commands[loaded_length: loaded_length + episode_length] = np_commands
#     expert_speeds[loaded_length: loaded_length + episode_length] = np_speeds
#     expert_actions[loaded_length: loaded_length + episode_length] = np_actions

#     loaded_length += episode_length

# print('data loaded!')



In [ ]:
# # np.save('./save_data/expert_states_front_view_2.npy',expert_states,)
# # np.save('./save_data/expert_commands_front_view_2.npy',expert_commands)
# # np.save('./save_data/expert_speeds_front_view_2.npy',expert_speeds)
# # np.save('./save_data/expert_actions_front_view_2.npy',expert_actions)

# expert_states=np.load('./save_data/expert_states_front_view_2.npy')
# expert_commands=np.load('./save_data/expert_commands_front_view_2.npy')
# expert_speeds=np.load('./save_data/expert_speeds_front_view_2.npy')
# expert_actions=np.load('./save_data/expert_actions_front_view_2.npy')

In [ ]:
# ratio = 0.85
# num_states = len(expert_states)
# expert_states_train = expert_states[: int(ratio * num_states)]
# expert_commands_train = expert_commands[: int(ratio * num_states)]
# expert_speeds_train = expert_speeds[: int(ratio * num_states)]
# expert_actions_train = expert_actions[: int(ratio * num_states)]
# expert_states_val = expert_states[int(ratio * num_states):]
# expert_commands_val = expert_commands[int(ratio * num_states):]
# expert_speeds_val = expert_speeds[int(ratio * num_states):]
# expert_actions_val = expert_actions[int(ratio * num_states):]
# print("train_data: ",len(expert_states_train))
# print("test_data: ",len(expert_states_val))

# 데이터 전처리

In [ ]:
# steering_angles = expert_actions_train[:, 1]
# throttle_data = expert_actions_train[:, 0]
# break_data = expert_actions_train[:, 2]
# expert_states = expert_states_train  # 이미지 데이터 (전체 데이터)
# labels=steering_angles = expert_actions_train[:, 1]
# labels = np.round(labels, decimals=3)
# label_counts=Counter(labels)

# sorted_counts = sorted(label_counts.items(), key=lambda x: x[1], reverse=True)
# # 상위 60% 레이블만 선택
# top_60_percent_count = int(len(sorted_counts) * 0.6)
# top_60_percent_labels = sorted_counts[:top_60_percent_count]
# min_value_label = min(top_60_percent_labels, key=lambda x: x[1])

# print("label_counts: ",label_counts)
# # min_vlaue=min(label_counts.values())
# # print("최소 카운트: ",min_vlaue)
# # print("길이: ", len(label_counts.values()))

# # 결과 출력
# print("top_60_percent_count: ",top_60_percent_count)
# print("top_60_percent_labels: ",top_60_percent_labels)
# print("상위 60% 레이블 중 최소 갯수를 가진 레이블과 값: ", min_value_label)
# print("해당 레이블의 값: ", min_value_label[0])
# print("해당 레이블의 갯수: ", min_value_label[1])

In [ ]:
steering_angles = expert_actions_train[:, 1]
throttle_data = expert_actions_train[:, 0]
break_data = expert_actions_train[:, 2]
expert_states = expert_states_train  # 이미지 데이터 (전체 데이터)
labels=steering_angles = expert_actions_train[:, 1]
labels = np.round(labels, decimals=3)
label_counts=Counter(labels)
print("label_counts: ",label_counts)
min_vlaue=min(label_counts.values())
print("최소 카운트: ",min_vlaue)
print("길이: ", len(label_counts.values()))
cut_count=5
min_count=30 # 최소 카운트 100개
max_count=400
erase_per=60
balanced_indices = []

for label,count in tqdm(label_counts.items()):

    # if count<cut_count:
    #     pass
    # else:
    #     label_indices=np.where(labels==label)[0]
    #     if count<min_count:
    #         sampled_indices = np.random.choice(label_indices, min_count, replace=True)
    #     else:
    #         sampled_indices =np.random.choice(label_indices, max_count, replace=True)
    if count>cut_count:
        label_indices=np.where(labels==label)[0]
        if max_count>count:
            sampled_indices =np.random.choice(label_indices, count, replace=True)
        else:
            sampled_indices =np.random.choice(label_indices, max_count, replace=True)
    #     if label<0.03 and label>-0.03:
    #         k=int(count*(erase_per)/100)
    #         t=count-k
    #         sampled_indices = np.random.choice(label_indices, t, replace=True)
    #     else:
    #         sampled_indices =np.random.choice(label_indices, count, replace=True)
        balanced_indices.extend(sampled_indices)
index=sorted(balanced_indices)

balanced_expert_states = np.array(expert_states[index]) # 선택된 인덱스에 맞는 이미지 추출
balanced_throttle_data = np.array(throttle_data[index]).reshape(-1,1)
balanced_break_data = np.array(break_data[index]).reshape(-1,1)
balanced_steering_angles = np.array(labels[index]).reshape(-1,1)
balanced_speed=np.array(expert_speeds_train[index])
balanced_command=np.array(expert_commands_train[index])

action = np.concatenate((balanced_throttle_data, balanced_steering_angles, balanced_break_data), axis=1)
print(action.shape)


In [ ]:
# np.save('./save_data/norm_steering_expert_states_5cut_400_round3_v5.npy',balanced_expert_states)
# np.save('./save_data/norm_steering_expert_action_5cut_400_round3_v5.npy',action)
# np.save('./save_data/norm_steering_expert_speed_5cut_400_round3_v5.npy',balanced_speed)
# np.save('./save_data/norm_steering_expert_command_5cut_400_round3_v5.npy',balanced_command)

In [ ]:
#5cut_4oo round3_v4 x 지그재그로 가는듯..? 스티어링만 0.0018이 최대 90x256
#5cut_400_round4_v4
#5cut_200_round4_v4 지금

In [ ]:
# balanced_steering_angles_counts=Counter(balanced_steering_angles.reshape(-1))
# print("balanced_steering_angles_counts: ",balanced_steering_angles_counts)
# min_vlaue=min(balanced_steering_angles_counts.values())
# print("최소 카운트: ",min_vlaue)
# print("길이: ", len(balanced_steering_angles_counts.values()))

In [ ]:
expert_states_train=np.load('./save_data/norm_steering_expert_states_5cut_400_round3_v5.npy')
expert_actions_train=np.load('./save_data/norm_steering_expert_action_5cut_400_round3_v5.npy')
expert_speeds_train=np.load('./save_data/norm_steering_expert_speed_5cut_400_round3_v5.npy')
expert_commands_train=np.load('./save_data/norm_steering_expert_command_5cut_400_round3_v5.npy')

expert_states_val=np.load('./save_data/expert_states_val.npy')
expert_actions_val=np.load('./save_data/expert_actions_val.npy')
expert_speeds_val=np.load('./save_data/expert_speeds_val.npy')
expert_commands_val=np.load('./save_data/expert_commands_val.npy')

In [ ]:
plt.plot(expert_actions_train[:,0])

In [ ]:
import torchvision
class RandomTranslate(object):
    def __call__(self, image, steering_angle, range_x=80, range_y=20):

        trans_x = range_x * (np.random.rand() - 0.5)
        trans_y = range_y * (np.random.rand() - 0.5)
        
        #steering_angle += trans_x * 0.002
        #trans_m = torch.tensor([[1, 0, trans_x], [0, 1, trans_y]])
        #height, width = image.shape[1], image.shape[2]
        affine_transform = T.functional.affine(image, angle=0, translate=(trans_x, trans_y), scale=1.0, shear=[0, 0])
        return affine_transform, steering_angle
class RandFlip(object):
    
    def __call__(self, image,action):
        if random.randint(0, 1) == 1:
            image = torch.flip(image, [-1])  
            action = -action
        
        return image,action
    
class RandBrightness(object):

    def __call__(self, image,mode):
        img = image# 이미지를 [H, W, C]에서 [C, H, W]로 변환
    
            
        if mode=='channel_9' or mode=='concat':
            num=random.randint(0,3)
            if num==0:
                img[:3,:,:] = torchvision.transforms.functional.adjust_brightness(img[:3,:,:], 1 + np.random.uniform(-0.5, 0.5))
            elif num==1:
                
                img[3:6,:,:] = torchvision.transforms.functional.adjust_brightness(img[3:6,:,:], 1 + np.random.uniform(-0.5, 0.5))
            elif num==2:
                
                img[6:,:,:] = torchvision.transforms.functional.adjust_brightness(img[6:,:,:], 1 + np.random.uniform(-0.5, 0.5))
            img=torch.clamp(img,min=0,max=1)
        return img
class RandRotateView(object):
    def __call__(self, image,action,target_mode):
        if random.randint(0, 3) == 1 and abs(action) < 0.3:
            rotate = random.uniform(-30, 30)
            delta_angle = rotate / 60
            image = torchvision.transforms.functional.rotate(image, rotate)  # ndimage.rotate 대신 torchvision의 rotate 사용

            if target_mode == 'steer':
                action += delta_angle
        return image, action
    
class RandRotation(object):
    def __call__(self, image):
        rotate = random.uniform(-1, 1)
        image = torchvision.transforms.functional.rotate(image, rotate)
        image = torch.clip(image, 0, 1)
        return image

class CustomDataset(Dataset):
    def __init__(self, image, action,speed,command,transforms=None,mode='center',target_mode='steer',train_mode=True):
        self.image = image
        self.action = action
        self.command=command
        self.transforms = transforms
        self.mode=mode
        self.target_mode=target_mode
        self.train_mode=train_mode
        self.speed=torch.FloatTensor(speed)
        self.RandFlip=RandFlip()
        self.RandBrightness=RandBrightness()
        self.RandRotation=RandRotation()
        self.RandRotateView=RandRotateView()
        self.RandomTranslate=RandomTranslate()
        #self.resize=torchvision.transforms.Resize((320,320))

    def __getitem__(self, index):
        image=self.image[index].transpose(1,2,0)/255.
        image=image[22:,:,:]
        if self.target_mode=='steer':
            label=self.action[index][1]
            label=np.round(label,3)
            label=torch.FloatTensor([label])
            throttle=self.action[index][0]
            throttle=torch.FloatTensor([throttle])
            command=torch.FloatTensor(self.command[index])
        # elif self.target_mode=='throttle':
        elif self.target_mode=='breake':
            label=self.action[index][2]

            label=torch.FloatTensor([label])
        # left=image[:,:,:3]
        # center=image[:,:,3:6]
        # right=image[:,:,6:]
        
        
        if self.mode=='center':
            center=image[:,:,3:6]

            concat_image=center
            if self.train_mode==True:
                concat_image,label=self.RandFlip(concat_image,label)
                concat_image=self.RandBrightness(concat_image,self.mode)
                concat_image=self.RandRotation(concat_image,)
                concat_image,label=self.RandRotateView(concat_image,label,self.target_mode)
        elif self.mode=='channel_9' or self.mode=='concat':
            if self.train_mode==True:
                image = self.transforms(image=image)['image']
                image,label=self.RandFlip(image,label)
                image=self.RandBrightness(image,self.mode)
                image=self.RandRotation(image)
                image,label=self.RandRotateView(image,label,self.target_mode)
                image,label=self.RandomTranslate(image,label)
                #image=self.resize(image)
            elif self.train_mode==False:
                image = self.transforms(image=image)['image']

                # print(label)
                #image=self.resize(image)
            

        

        if self.transforms is not None:
            if self.mode=='total':
                
                if self.train_mode==True:
                    image = self.transforms(image=image)['image']
                    image=torch.cat((image[:3,:,:],image[3:6,:,:],image[6:,:,:]),dim=-1)
                    
                   
                    image,label=self.RandFlip(image,label)
                    image=self.RandBrightness(image,self.mode)
                    image=self.RandRotation(image)
                    image,label=self.RandRotateView(image,label,self.target_mode)
                    #image=self.resize(image)
                    

                elif self.train_mode==False:
                    #label=label-steering_min/steering_max-steering_min
                    image = self.transforms(image=image)['image']
                    image=torch.cat((image[:3,:,:],image[3:6,:,:],image[6:,:,:]),dim=-1)
                    #image=self.resize(image)
                    

        
        if self.action is not None:
            s=self.speed[index]
            image=image
            steering=torch.round(label * 1000) / 1000 #3자리수에서 올림
            label=torch.cat((throttle,steering),dim=-1)

            return image,s,command,label
        else:
            return image
    
    def __len__(self):
        return len(self.image)

In [ ]:

train_transform=A.Compose([
    A.GaussianBlur(blur_limit=(3, 5), p=0.3),  
    ToTensorV2()
])
test_transform=A.Compose([
    #A.Resize(320,320),
   
    ToTensorV2()
])
target_label_mode='steer'
camera_view='channel_9' #channel_9, total,center,concat
train_transform_dataset=CustomDataset(expert_states_train,expert_actions_train,expert_speeds_train,expert_commands_train,train_transform,mode=camera_view,target_mode=target_label_mode,train_mode=True)
val_transform_dataset=CustomDataset(expert_states_val,expert_actions_val,expert_speeds_val,expert_commands_val,test_transform,mode=camera_view,target_mode=target_label_mode,train_mode=False)
train_loader=DataLoader(train_transform_dataset,batch_size=CFG['BATCH_SIZE'],shuffle=True)
test_loader=DataLoader(val_transform_dataset,batch_size=1,shuffle=False)

In [ ]:
print(len(train_loader))
image,speed,command,label=train_transform_dataset.__getitem__(300)
print(label)
print(speed)
print(command)
print(image.shape)
plt.subplot(3,1,1)
plt.imshow(image[0:3,:,:].permute(1,2,0))
plt.subplot(3,1,2)
plt.imshow(image[3:6,:,:].permute(1,2,0))
plt.subplot(3,1,3)
plt.imshow(image[6:9,:,:].permute(1,2,0))




In [ ]:
# train_loader=DataLoader(train_transform_dataset,batch_size=1,shuffle=True)
# test=torch.tensor([0,0,0])
# for image,speed,command,label in tqdm(train_loader):
#     branch_idx = torch.argmax(command, dim=1)[0]
#     test[branch_idx]+=1


In [ ]:

class BaseModel(nn.Module):
    def __init__(self,model_name,class_number=1):
        super(BaseModel,self).__init__()
        
        self.conv1=nn.Conv2d(9,3,kernel_size=3,stride=1,padding=1)
        self.batch_norm=nn.BatchNorm2d(3)
        self.relu=nn.ReLU()

       
        self.basemodel=timm.create_model(model_name,pretrained=False,num_classes=512)
        self.tanh=nn.Tanh()
        self.dropout=nn.Dropout(0.3)


        ###############Mesaurement###############

        self.measure_fc1=nn.Linear(1,128)
        self.measure_fc2=nn.Linear(128,128)
        self.measure_fc3=nn.Linear(128,128)

        self.steering_branches = nn.ModuleList([
            nn.Sequential(
                nn.Linear(512 + 128, 256),
                nn.Tanh(),
                nn.Dropout(0.3),
                nn.Linear(256, 256),
                nn.Tanh(),
                nn.Dropout(0.3),
                nn.Linear(256, class_number)
            ) for _ in range(3)  # 3개의 브랜치로 수정
        ])

        self.throttle_branches = nn.ModuleList([
            nn.Sequential(
                nn.Linear(512 + 128, 256),
                nn.ReLU(),
                nn.Dropout(0.3),
                nn.Linear(256, 256),
                nn.ReLU(),
                nn.Dropout(0.3),
                nn.Linear(256, class_number)

            ) for _ in range(3)  # 3개의 브랜치로 수정
        ])

        

       
        
    def forward(self,input,speed,command):
        
        x=self.conv1(input)
        x=self.batch_norm(x)
        x=self.relu(x)

        x=self.basemodel(x)

        s=self.dropout(self.relu(self.measure_fc1(speed)))
        s=self.dropout(self.relu(self.measure_fc2(s)))
        s=self.dropout(self.relu(self.measure_fc3(s)))
       
        input_cat=torch.cat((x,s),dim=-1)
        branch_idx = torch.argmax(command, dim=1)
        #print(branch_idx)

        steering = torch.zeros((input.size(0), 1)).to(input.device)
        throttle = torch.zeros((input.size(0), 1)).to(input.device)

        for i in range(input.size(0)):  # batch 내의 각 샘플에 대해 적절한 branch 선택
            steering[i] = self.steering_branches[branch_idx[i]](input_cat[i].unsqueeze(0))
            steering_dist =self.steering_branches[branch_idx[i]][:5](input_cat[i].unsqueeze(0))
            throttle[i] = self.throttle_branches[branch_idx[i]](input_cat[i].unsqueeze(0))
            throttle_dist = self.throttle_branches[branch_idx[i]][:5](input_cat[i].unsqueeze(0))
            dist=torch.cat((steering_dist,throttle_dist),dim=-1)
        
        

    
        return throttle, steering,steering_dist,throttle_dist
       

In [ ]:
def RMSE(pred,label):
    c=nn.MSELoss().to(CFG['device'])
    loss=torch.sqrt(c(pred,label))
    return loss
def I_FGSM(target_model,target,image,speed,command,target_point,epsilon,device):
    image=image.clone().detach().to(device)
    speed=speed.clone().detach().to(device)
    command=command.clone().detach().to(device)
    y_throttle,y_steering,_,_=target_model(image,speed,command)
    


    if target_point=='throttle':
        y_target=y_throttle+target
    elif target_point=='steering':
        y_target=y_steering+target
    else:
        y_throttle_target=y_throttle+target
        y_steering_target=y_steering+target

    ori_image =image.clone().detach()
    ori_speed = speed.clone().detach()
    ori_command = command.clone().detach()

    for _ in range(5):
        image.requires_grad=True
        y_throttle,y_steering,_,_=target_model(image,speed,command)
        y_adv_throttle=y_throttle
        y_adv_steering=y_steering
        if target_point=='throttle':
            loss_y = F.mse_loss(y_adv_throttle, y_target)
        elif target_point=='steering':
            loss_y = F.mse_loss(y_adv_steering, y_target)
        else:
            loss_y_throttle= F.mse_loss(y_adv_throttle, y_throttle_target)
            loss_y_steering = F.mse_loss(y_adv_steering, y_steering_target)
            loss_y=(loss_y_throttle+loss_y_steering)
            #print(loss_y)
        grad = torch.autograd.grad(
                loss_y, image, retain_graph=False, create_graph=False
            )[0]
        

        adv_images = image + (2/255)* grad.sign()
        a = torch.clamp(ori_image - epsilon, min=0)
        b = (adv_images >= a).float() * adv_images + (
                adv_images < a
            ).float() * a  # nopep8
        c = (b > ori_image + epsilon).float() * (ori_image + epsilon) + (
                b <= ori_image + epsilon
            ).float() * b  # nopep8
        image = torch.clamp(c, max=1).detach()
    return image

def PGD(target_model,target,image,speed,command,target_point,epsilon,device):
    image=image.clone().detach().to(device)
    speed=speed.clone().detach().to(device)
    command=command.clone().detach().to(device)
    y_throttle,y_steering,_,_=target_model(image,speed,command)

    adv_image =image.clone().detach()
    adv_speed = speed.clone().detach()
    adv_command = command.clone().detach()

    if target_point=='throttle':
        y_target=y_throttle+target
    elif target_point=='steering':
        y_target=y_steering+target
    else:
        y_throttle_target=y_throttle+target
        y_steering_target=y_steering+target

    for _ in range(5):
        adv_image.requires_grad=True
        y_adv_throttle,y_adv_steering,_,_=target_model(adv_image,adv_speed,adv_command)

        if target_point=='throttle':
            loss_y = F.mse_loss(y_adv_throttle, y_target)
        elif target_point=='steering':
            loss_y = F.mse_loss(y_adv_steering, y_target)
        else:
            loss_y_throttle= F.mse_loss(y_adv_throttle, y_throttle_target)
            loss_y_steering = F.mse_loss(y_adv_steering, y_steering_target)
            loss_y=(loss_y_throttle+loss_y_steering)

        grad = torch.autograd.grad(
                loss_y, adv_image, retain_graph=False, create_graph=False
            )[0]
        adv_image = adv_image.detach() + (2/255) * grad.sign()
        delta = torch.clamp(adv_image - image, min=-epsilon, max=epsilon)
        adv_image = torch.clamp(image + delta, min=0, max=1).detach()
    return adv_image

In [28]:
model_name=['convit_base']#'mobilenetv2_100'convit_basevgg16

#model_name=['mobilenetv2_100']

train_mode=0 #    0:   일반 학습   1:     advtraining
target_point='throttle&steering'
if train_mode==0:

    for model_n in model_name:
        print("MODEL_NAME: ",model_n)
        print("MODE: ",camera_view)
        
        model=BaseModel(model_name=model_n).to(CFG['device'])

        flag_loss=10000
        count=0
        patient=500

        
        optimizer=torch.optim.Adam(model.parameters(),lr= 0.00005,betas=(0.5, 0.999), amsgrad=True)
        #optimizer=torch.optim.SGD(model.parameters(),lr= 0.0003)
        scedular = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=50)
        #criterion=nn.L1Loss().to(CFG['device'])
        criterion=nn.MSELoss().to(CFG['device'])
        test_criterion=nn.MSELoss().to(CFG['device'])
        Mae_criterion=nn.L1Loss().to(CFG['device'])

        
        for epoch in range(1,CFG['EPOCH']+1):
            model.train()
            train_loss=[]
            test_loss=[]
            test_loss_MAE=[]
            test_loss_RMSE=[]
            total_throttle_loss=0.0
            total_steering_loss=0.0
            for image,speed,command,label in tqdm(train_loader):
                img=image.to(CFG['device'])
                labels=label.to(CFG['device'])
                speeds=speed.to(CFG['device'])
                command=command.to(CFG['device'])

                optimizer.zero_grad()
                output_throttle,output_steering,_,_=model(img,speeds,command)
                throttle_loss=RMSE(output_throttle,labels[:,0].unsqueeze(1))
                steering_loss=RMSE(output_steering,labels[:,1].unsqueeze(1))
                total_steering_loss+=steering_loss.cpu().detach()
                total_throttle_loss+=throttle_loss.cpu().detach()
                loss=(throttle_loss*0.4+steering_loss*0.6)
                loss.backward()
                optimizer.step()

                train_loss.append(loss.item())
            tr_loss=np.mean(train_loss)

            if epoch%10==1:
                with torch.no_grad():
                    throttle_output_list=[]
                    throttle_real_list=[]

                    steering_output_list=[]
                    steering_real_list=[]
                    model.eval()
                    for test_image,test_speed,test_command,test_label in tqdm(test_loader):
                        test_img=test_image.to(CFG['device'])
                        test_speeds=test_speed.to(CFG['device'])
                        test_labels=test_label.to(CFG['device'])
                        test_command=test_command.to(CFG['device'])
                        throttle_real_list.append(test_labels[0][0].item())
                        steering_real_list.append(test_labels[0][1].item())

                        test_output_throttle,test_output_steering,_,_=model(test_img,test_speeds,test_command)
                        throttle_output_list.append(test_output_throttle[0].item())
                        steering_output_list.append(test_output_steering[0].item())
                        ############################################
                        test_losses_throttle=test_criterion(test_output_throttle,test_labels[:,0].unsqueeze(1))
                        MAE_test_throttle=Mae_criterion(test_output_throttle,test_labels[:,0].unsqueeze(1))
                        RMSE_loss_throttle=RMSE(test_output_throttle,test_labels[:,0].unsqueeze(1).unsqueeze(1))

                        test_losses_steering=test_criterion(test_output_steering,test_labels[:,1].unsqueeze(1))
                        MAE_test_steering=Mae_criterion(test_output_steering,test_labels[:,1].unsqueeze(1))
                        RMSE_loss_steering=RMSE(test_output_steering,test_labels[:,1].unsqueeze(1))
                        ############################################
                        test_losses=(test_losses_throttle*0.4+test_losses_steering*0.6)
                        MAE_test=(MAE_test_throttle*0.4+MAE_test_steering*0.6)
                        RMSE_loss=(RMSE_loss_throttle*0.4+RMSE_loss_steering*0.6)

                        test_loss_MAE.append(MAE_test.item())
                        test_loss_RMSE.append(RMSE_loss.item())
                        test_loss.append(test_losses.item())
                t_loss=np.mean(test_loss)
                MAE_loss=np.mean(test_loss_MAE)
                RMSE_loss=np.mean(test_loss_RMSE)
                curve_lr=optimizer.param_groups[0]['lr']
                #plot_durations(real_list,output_list)

                print(f"EPOCH:{epoch},Train_LOSS:{tr_loss:.5f} VAL_LOSS:{t_loss:.5f} CURVE_LOSS:{curve_lr:.9f}") 
                print(f"label throttle: {test_labels[0][0].item():.5f} predict throttle: {test_output_throttle[0].item():.5f}")
                print(f"label steering: {test_labels[0][1].item():.5f} predict steering: {test_output_steering[0].item():.5f}")
                print(f"MAE: {MAE_loss:.6f}  RMSE: {RMSE_loss:.6f} count: {count}")
                print(f"train_throttle: {(total_throttle_loss/len(train_loader)):.6f} train_steering: {(total_steering_loss/len(train_loader)):.6f}")
                plt.plot(steering_real_list)
                plt.plot(steering_output_list)
                plt.savefig('./'+model_n+'_steering.jpg', format='jpg')
                plt.clf()
                plt.plot(throttle_real_list)
                plt.plot(throttle_output_list)
                plt.savefig('./'+model_n+'_throttle.jpg', format='jpg')
                plt.clf()
                if t_loss<flag_loss:
                    torch.save(model,'./model_list/'+str(epoch)+'_'+model_n+'_'+camera_view+'_'+target_label_mode+'_'+str(t_loss)[:8]+'.pt')
                    flag_loss=t_loss
                    count=0
                else:
                    count+=10
            else:
                print(f"train_throttle: {(total_throttle_loss/len(train_loader)):.6f} train_steering: {(total_steering_loss/len(train_loader)):.6f}")
            if count>patient:
                torch.save(model,'./model_list/last_'+model_n+'_'+camera_view+'_'+target_label_mode+'_'+str(t_loss)[:8]+'.pt')
                break
            scedular.step(tr_loss)
        torch.save(model,'./model_list/last_'+model_n+'_'+camera_view+'_'+target_label_mode+'_'+str(t_loss)[:8]+'.pt')

elif train_mode==1: # 1: 적대적 학습
    
    for model_n in model_name:
        print("MODEL_NAME: ",model_n)
        print("MODE: ",camera_view)
        
        advtraining_net=BaseModel(model_name=model_n).to(CFG['device'])
        #distillation_net=torch.load('F:/VSC/graduate_for_simulator/model_list/last_distillation_mobilenetv2_100_channel_9_steer_0.002392.pt').to(CFG['device'])
        optimizer=torch.optim.Adam(advtraining_net.parameters(),lr= 0.00005,betas=(0.5, 0.999), amsgrad=True)
        scedular = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=50)
        #criterion=nn.L1Loss().to(CFG['device'])
        criterion=nn.MSELoss().to(CFG['device'])
        test_criterion=nn.MSELoss().to(CFG['device'])
        Mae_criterion=nn.L1Loss().to(CFG['device'])

        if model_n=='mobilenetv2_100':
            t_model=torch.load('F:/VSC/graduate_for_simulator/model_list/best_model/901_mobilenetv2_100_channel_9_steer_0.002354.pt').to(CFG['device'])
        elif model_n=='resnet18':
            t_model=torch.load('F:/VSC/graduate_for_simulator/model_list/best_model/601_resnet18_channel_9_steer_0.002256.pt').to(CFG['device'])
        elif model_n=='vgg16':
            t_model=torch.load('F:/VSC/graduate_for_simulator/model_list/best_model/61_vgg16_channel_9_steer_0.002720.pt').to(CFG['device'])

            
        flag_loss=10000
        count=0
        patient=500

        
        for epoch in range(1,CFG['EPOCH']+1):

            train_loss=[]
            test_loss=[]
            test_loss_MAE=[]
            test_loss_RMSE=[]

            

            total_throttle_loss=0.0
            total_steering_loss=0.0
            total_dist_loss=0.0
            for image,speed,command,label in tqdm(train_loader):
                img=image.to(CFG['device'])
                labels=label.to(CFG['device'])
                speeds=speed.to(CFG['device'])
                command=command.to(CFG['device'])
                optimizer.zero_grad()

                test_img=img[:img.shape[0]//2]
                test_speeds=speeds[:img.shape[0]//2]
                test_command=command[:img.shape[0]//2]

                perturbed_image=PGD(target_model=advtraining_net,
                                        target=0.3,
                                        image=test_img,
                                        speed=test_speeds,
                                        command=test_command,
                                        target_point=target_point,
                                        epsilon=30/255,
                                        device=CFG['device'])
                
                img[:img.shape[0]//2]=perturbed_image


                output_throttle,output_steering,_,_=advtraining_net(img,speeds,command)
    

                throttle_loss=RMSE(output_throttle,labels[:,0].unsqueeze(1))
                steering_loss=RMSE(output_steering,labels[:,1].unsqueeze(1))

    

                
                total_steering_loss+=steering_loss.cpu().detach()
                total_throttle_loss+=throttle_loss.cpu().detach()
                loss=(throttle_loss*0.4+steering_loss*0.6)
                loss.backward()
                optimizer.step()

                train_loss.append(loss.item())
            tr_loss=np.mean(train_loss)


            if epoch%10==1:
                with torch.no_grad():
                    throttle_output_list=[]
                    throttle_real_list=[]

                    steering_output_list=[]
                    steering_real_list=[]
                    advtraining_net.eval()
                    for test_image,test_speed,test_command,test_label in tqdm(test_loader):
                        test_img=test_image.to(CFG['device'])
                        test_speeds=test_speed.to(CFG['device'])
                        test_labels=test_label.to(CFG['device'])
                        test_command=test_command.to(CFG['device'])
                        throttle_real_list.append(test_labels[0][0].item())
                        steering_real_list.append(test_labels[0][1].item())

                        test_output_throttle,test_output_steering,_,_=advtraining_net(test_img,test_speeds,test_command)
                        throttle_output_list.append(test_output_throttle[0].item())
                        steering_output_list.append(test_output_steering[0].item())
                        ############################################
                        test_losses_throttle=test_criterion(test_output_throttle,test_labels[:,0].unsqueeze(1))
                        MAE_test_throttle=Mae_criterion(test_output_throttle,test_labels[:,0].unsqueeze(1))
                        RMSE_loss_throttle=RMSE(test_output_throttle,test_labels[:,0].unsqueeze(1).unsqueeze(1))

                        test_losses_steering=test_criterion(test_output_steering,test_labels[:,1].unsqueeze(1))
                        MAE_test_steering=Mae_criterion(test_output_steering,test_labels[:,1].unsqueeze(1))
                        RMSE_loss_steering=RMSE(test_output_steering,test_labels[:,1].unsqueeze(1))
                        ############################################
                        test_losses=(test_losses_throttle*0.4+test_losses_steering*0.6)
                        MAE_test=(MAE_test_throttle*0.4+MAE_test_steering*0.6)
                        RMSE_loss=(RMSE_loss_throttle*0.4+RMSE_loss_steering*0.6)

                        test_loss_MAE.append(MAE_test.item())
                        test_loss_RMSE.append(RMSE_loss.item())
                        test_loss.append(test_losses.item())
                t_loss=np.mean(test_loss)
                MAE_loss=np.mean(test_loss_MAE)
                RMSE_loss=np.mean(test_loss_RMSE)
                curve_lr=optimizer.param_groups[0]['lr']
                #plot_durations(real_list,output_list)

                print(f"EPOCH:{epoch},Train_LOSS:{tr_loss:.5f} VAL_LOSS:{t_loss:.5f} CURVE_LOSS:{curve_lr:.9f}") 
                print(f"label throttle: {test_labels[0][0].item():.5f} predict throttle: {test_output_throttle[0].item():.5f}")
                print(f"label steering: {test_labels[0][1].item():.5f} predict steering: {test_output_steering[0].item():.5f}")
                print(f"MAE: {MAE_loss:.6f}  RMSE: {RMSE_loss:.6f} count: {count}")
                print(f"train_throttle: {(total_throttle_loss/len(train_loader)):.6f} train_steering: {(total_steering_loss/len(train_loader)):.6f}")
                plt.plot(steering_real_list)
                plt.plot(steering_output_list)
                plt.savefig('./'+model_n+'_advtraining_steering.jpg', format='jpg')
                plt.clf()
                plt.plot(throttle_real_list)
                plt.plot(throttle_output_list)
                plt.savefig('./'+model_n+'_advtraining_throttle.jpg', format='jpg')
                plt.clf()
                if t_loss<flag_loss:
                    torch.save(advtraining_net,'./model_list/advtraining_'+str(epoch)+'_'+model_n+'_'+camera_view+'_'+target_label_mode+'_'+str(t_loss)[:8]+'.pt')
                    flag_loss=t_loss
                    count=0
                else:
                    count+=10
            else:
                print(f"train_throttle: {(total_throttle_loss/len(train_loader)):.6f} train_steering: {(total_steering_loss/len(train_loader)):.6f} train_dist: {(total_dist_loss/len(train_loader)):.6f}")
            
            # if epoch%10==1:
            #     test_loss_MAE=[]
            #     test_loss_MSE=[]
            #     test_loss_RMSE=[]

            #     mse_criterion=nn.MSELoss().to(CFG['device'])
            #     mae_criterion=nn.L1Loss().to(CFG['device'])
            #     real_label=[]
            #     predict_adv_label=[]
            #     predict_label=[]
            #     total_diff = np.array([])
            #     attack_method='PGD'#opt,MI-FGSM, I-FGSM,PGD,CW,None
            #     attack_point=2 #0: throttle 1: steering 2: throttle&steering


            #     Transfer_attack=False

            #     feature_squeezing_flag=False
            #     gradient_mode=False
                


                
            #     main_model=advtraining_net.eval()
            #     t_model=t_model.eval()
            #     for i,(test_image,test_speed,test_command,test_label) in enumerate(tqdm(test_loader)):
                    
            #         test_img=test_image.to(CFG['device'])
            #         test_speeds=test_speed.to(CFG['device'])
            #         test_labels=test_label.to(CFG['device'])
            #         test_command=test_command.to(CFG['device'])
            #         #print(test_labels[0][1])

            #         if attack_method=='PGD':
            #             perturbed_image=PGD(target_model=main_model,
            #                                     target=0.3,
            #                                     image=test_img,
            #                                     speed=test_speeds,
            #                                     command=test_command,
            #                                     target_point=target_point,
            #                                     epsilon=8/255,
            #                                     device=CFG['device'])
                    

            #         test_adv_output_throttle,test_adv_output_steering,_,_=main_model(perturbed_image,test_speeds,test_command) #어택

            #         test_output_throttle,test_output_steering,_,_=t_model(test_img,test_speeds,test_command) #노말
                    
            #         if attack_point==0:#0: throttle
            #             throttle_diff = (test_adv_output_throttle - test_output_throttle).detach().cpu().numpy()
            #             diff = np.squeeze((throttle_diff))
            #         elif attack_point==1:#1: steering
            #             steering_diff = (test_adv_output_steering - test_output_steering).detach().cpu().numpy() 	
            #             diff = np.squeeze((steering_diff))
            #         else:#2:throttle&steering
            #             throttle_diff = (test_adv_output_throttle - test_output_throttle).detach().cpu().numpy()
            #             steering_diff = (test_adv_output_steering - test_output_steering).detach().cpu().numpy() 
            #             diff = np.squeeze((steering_diff+throttle_diff))



            #         RMSE_throttle_loss=RMSE(test_adv_output_throttle,test_labels[:,0].unsqueeze(1))
            #         RMSE_steering_loss=RMSE(test_adv_output_steering,test_labels[:,1].unsqueeze(1))
            #         RMSE_loss=(RMSE_throttle_loss*0.4+RMSE_steering_loss*0.6)
            #         test_loss_RMSE.append(RMSE_loss.item())

            #         MSE_throttle_loss=mse_criterion(test_adv_output_throttle,test_labels[:,0].unsqueeze(1))
            #         MSE_steering_loss=mse_criterion(test_adv_output_steering,test_labels[:,1].unsqueeze(1))
            #         MSE_loss=(MSE_throttle_loss*0.4+MSE_steering_loss*0.6)
            #         test_loss_MSE.append(MSE_loss.item())

            #         MAE_throttle_loss=mae_criterion(test_adv_output_throttle,test_labels[:,0].unsqueeze(1))
            #         MAE_steering_loss=mae_criterion(test_adv_output_steering,test_labels[:,1].unsqueeze(1))
            #         MAE_loss=(MAE_throttle_loss*0.4+MAE_steering_loss*0.6)
            #         test_loss_MAE.append(MAE_loss.item())

            #         total_diff = np.concatenate((total_diff, [diff]))	


            #         predict_adv_label.append(test_adv_output_steering[0][0].cpu().detach())
            #         predict_label.append(test_output_steering[0][0].cpu().detach())
            #         if i%100==0:
            #             MSE_loss=np.mean(test_loss_MSE)
            #             MAE_loss=np.mean(test_loss_MAE)
            #             RMSE_loss=np.mean(test_loss_RMSE)
            #             success_ = len(total_diff[abs(total_diff) >= 0.3]) 
            #             print("진짜예측과 가짜 예측의 차이 평균: ", np.mean(total_diff))
            #             print("공격 성공: ", success_/(i+1)*100)
            #             print("RMSE: ",RMSE_loss)
            #             print("MSE: ",MSE_loss)
            #             print("MAE: ",MAE_loss)
            #             print("=====================================================")
            #             break
                    
            
            if count>patient:
                torch.save(advtraining_net,'./model_list/last_advtraining_'+model_n+'_'+camera_view+'_'+target_label_mode+'_'+str(t_loss)[:8]+'.pt')
                break
            scedular.step(tr_loss)
        torch.save(advtraining_net,'./model_list/last_advtraining_'+model_n+'_'+camera_view+'_'+target_label_mode+'_'+str(t_loss)[:8]+'.pt')

        
            
        






100%|██████████| 358/358 [07:11<00:00,  1.21s/it]


train_throttle: 0.059951 train_steering: 0.061633


100%|██████████| 358/358 [07:13<00:00,  1.21s/it]


train_throttle: 0.060038 train_steering: 0.061818


100%|██████████| 358/358 [07:11<00:00,  1.20s/it]


...

100%|██████████| 358/358 [05:54<00:00,  1.01it/s]


train_throttle: 0.058349 train_steering: 0.064574


  4%|▍         | 14/358 [00:13<05:40,  1.01it/s]


KeyboardInterrupt: 

<Figure size 640x480 with 0 Axes>

In [ ]:
advtraining_net=BaseModel(model_name=model_n).to(CFG['device'])
for image,speed,command,label in tqdm(train_loader):
        img=image.to(CFG['device'])
        labels=label.to(CFG['device'])
        speeds=speed.to(CFG['device'])
        command=command.to(CFG['device'])
        optimizer.zero_grad()

        test_img=img[:img.shape[0]//2]
        test_speeds=speeds[:img.shape[0]//2]
        test_command=command[:img.shape[0]//2]

        perturbed_image=PGD(target_model=advtraining_net,
                                target=0.3,
                                image=test_img,
                                speed=test_speeds,
                                command=test_command,
                                target_point=target_point,
                                epsilon=30/255,
                                device=CFG['device'])
        
        img[:img.shape[0]//2]



        output_throttle,output_steering,_,_=advtraining_net(img,speeds,command)
        break

In [ ]:
model=torch.load('F:/VSC/graduate_for_simulator/model_list/best_model/61_vgg16_channel_9_steer_0.002720.pt').to(CFG['device'])
test_loader=DataLoader(val_transform_dataset,batch_size=1,shuffle=False)
real_label=[]
predict_label=[]
t=[]
test_criterion=nn.MSELoss().to(CFG['device'])
# model=BaseModel(model_name='resnet18').to(CFG['device'])
with torch.no_grad():
    model.eval()
    for test_image,test_speed,test_command,test_label in tqdm(test_loader):
        
        test_img=test_image.to(CFG['device'])
        test_speeds=test_speed.to(CFG['device'])
        test_labels=test_label.to(CFG['device'])
        test_command=test_command.to(CFG['device'])

        
        test_output_throttle,test_output_steering=model(test_img,test_speeds,test_command)
        #loss=test_criterion(output,test_label)
        #t.append(loss.cpu().detach())
        predict_label.append(test_output_steering[0][0].cpu().detach())
        real_label.append(test_labels[0][1].cpu().detach())

In [ ]:
plt.figure(figsize=(20,10))
plt.plot(predict_label)
plt.plot(real_label)

In [ ]:
# image=test_image.to(CFG['device'])
# label=test_label.to(CFG['device'])
# output=model(image)
# print('out_put: ',output)
# print('label: ',label)
# print('loss: ',loss)